# Lecture 11

## Solving PDEs with the method of weighted residuals

Up until now we have considered how to approximate a generic function $f(x)$ in a function space $V_N = \text{span}\{\psi_j\}_{j=0}^N$. We have considered both variational formulations (Galerkin and least squares) and collocation (interpolation). But the underlying original problem has simply been to find some function $u \approx u_N = \sum_{j=0}^N\hat{u}_j \psi_j$ such that $u = f$.

We will now take it one step further and solve ordinary differential equations with a method that is almost identical. Instead of finding just $u=f$, we will now attempt to find the solution to

$$
\mathcal{L}(u) = f,
$$(eq-L-op)

where $\mathcal{L}$ is a generic operator acting on the function $u$. Hence $\mathcal{L}(u)$  can by anything, including just $\mathcal{L}(u)=u$, and by {eq}`eq-L-op` we could for example mean any one of

$$
\begin{align*}
u  &= f, \\
u' &= f, \\
u'' &= f, \\
u'' +\alpha u' + \lambda u  &= f, \\
\frac{d}{dx}\left(\alpha \frac{d u}{dx}\right) &= f,
\end{align*}
$$

where $\alpha$ and $\lambda$ are either constant parameters of functions of $x$. From {eq}`eq-L-op` we can also define a residual (or an error)

$$
\mathcal{R} = \mathcal{L}(u)-f,
$$

that we ultimately want to be zero. If we now insert the approximation $u \approx u_N \in V_N$ (the same as we have used up until now) we can also define another residual

$$
\mathcal{R}_N = \mathcal{L}(u_N)-f.
$$

The method of weighted residuals (MWR) requires that this residual is orthogonal to some test space $W$

$$
(\mathcal{R}_N, v) = 0 \quad \forall \, v \in W.
$$

The Galerkin method is as such a MWR where $W = V_N$. Likewise, it can be shown that the least squares method 

$$
\frac{\partial (\mathcal{R_N}, \mathcal{R}_N)}{\partial \hat{u}_j} = 0 \quad j \in (0, 1, \ldots, N),
$$

can be written as

$$
\left(\mathcal{R}_N, \frac{\partial \mathcal{R}_N}{\partial \hat{u}_j}\right) \quad j \in (0, 1, \ldots, N).
$$

This means that also the LSM is a MWR where $W = \text{span}\{\frac{\partial \mathcal{R}_N}{\partial \hat{u}_j}\}_{j=0}^N$.

More surprisingly, the collocation method can also be considered a MWR. The collocation method is simply

$$
\mathcal{R}_N(x_j) = f(x_j), \quad j \in (0, 1, \ldots, N).
$$(eq-coll-R)

By choosing test functions $v = \delta(x-x_j)$, where $\delta(x)$ is Dirac's delta function, we can write this equivalently as a MWR:


$$
(\mathcal{R}_N, \delta(x-x_j)) = (f, \delta(x-x_j)), \quad j \in (0, 1, \ldots, N).
$$(eq-coll-Gal)

The equality of {eq}`eq-coll-R` with {eq}`eq-coll-Gal` follows since

$$
\int_{\Omega} \mathcal{R}_N(x) \delta(x-x_j) dx = \mathcal{R}_N(x_j) \, \text{  and  } \,  \int_{\Omega} f(x) \delta(x-x_j) dx = f(x_j).
$$

Lets consider a concrete example where

$$
u'' = f, \quad  u(-1) = u(1) = 0, \, x \in [-1, 1],
$$

such that $\mathcal{L}(u) = u''$. Note that there is one significant difference from merely function approximation, since the problem needs to be solved with boundary conditions! Since there are 2 derivatives, we need two boundary conditions and we have also prescribed homogeneous Dirichlet on both sides of the domain.

The boundary conditions can be incorporated into the trial space by choosing basis functions that all satisfy $\psi_j(-1) = \psi_j(1) = 0$. Possible basis functions are thus

$$
\begin{align*}
\psi_j(x) &= \sin\left(\pi (j+1) (x+1)/2\right), \\
\psi_j(x) &= P_j - P_{j+2}, \\
\psi_j(x) &= T_j - T_{j+2}, \\
\end{align*}
$$

where the latter two are zero on the boundaries because $P_j(-1) = T_j(-1) = (-1)^j$ and $P_j(1)=T_j(1) = 1$. The first sine function is a bit odd because we need to map the true domain $[-1, 1]$ to the sine function's computational domain, which is $[0, 1]$. 

We consider first the regular, unweighted Galerkin method in the $L^2([-1, 1])$ inner product space, and as such only the first two are relevant. The Chebyshev polynomials should only be used with a weighted $L^2_{\omega}([-1, 1])$ inner product.

Create $V_N = \text{span}\{\sin(\pi (j+1) (x+1)/2)\}_{j=0}^N$ and find $u_N \in V_N$ such that

$$
(u_N''-f, v) = 0, \quad \forall \, v \in V_N.
$$

Insert for $u_N$ and $v$ to obtain the linear algebra problem

$$
\sum_{j=0}^N \left(\psi''_j, \psi_i  \right) \hat{u}_j = (f, \psi_i), \quad i \in (0, 1, \ldots, N)
$$(eq-poisson-gal)

The matrix $S = (s_{ij})_{i,j=0}^N$, where $s_{ij}=(\psi''_j, \psi_i)$, is usually called the **stiffness matrix**. With the chosen basis functions we get

$$
\begin{align*}
s_{ij} &= ((\sin( \pi (j+1)  (x+1)/2))'', \sin(\pi (i+1)(x+1)/2)), \\
    &= -\frac{(j+1)^2\pi^2}{4} (\pi \sin((j+1)(x+1)/2), \sin(\pi (i+1) (x+1)/2)), \\
    &= -\frac{(j+1)^2 \pi^2}{4} \delta_{ij}.
\end{align*}
$$

And since this matrix is diagonal Eq. {eq}`eq-poisson-gal` is solved with

$$
\hat{u}_i = \frac{-4}{(i+1)^2 \pi^2}(f, \sin( \pi (i+1)(x+1)/2)).
$$

Lets check the accuracy using the method of manufactured solutions and choose $u(x) = (1-x^2)\exp(\cos(x))$, which satisfies the boundary conditions.

In [ ]:
import numpy as np
import sympy as sp
from scipy.integrate import quad
import matplotlib.pyplot as plt
from IPython.display import display

x = sp.Symbol('x')

ue = (1-x**2)*sp.exp(sp.cos(x-0.5))
f = ue.diff(x, 2)
#uhat = lambda j: -(4/(j+1)**2/sp.pi**2)*sp.integrate(f*sp.sin((j+1)*sp.pi*(x+1)/2), (x, -1, 1))
uhat = lambda j: -(4/(j+1)**2/np.pi**2)*quad(sp.lambdify(x, f*sp.sin((j+1)*sp.pi*(x+1)/2)), -1, 1)[0]

uh = []
N = 20
for k in range(N):
    uh.append(uhat(k))

M = 20
xj = np.linspace(-1, 1, M+1)
sines = np.sin(np.pi/2*(np.arange(len(uh))[None, :]+1)*(xj[:, None]+1))
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
ax1.plot(xj, sp.lambdify(x, ue)(xj), 'b',
         xj, sines[:, :1] @ np.array(uh)[:1], 'k:',
         xj, sines @ np.array(uh), 'r--')
ax2.loglog(abs(np.array(uh)))
ax1.legend(['Exact', 'One coefficient', f'{N} coefficients'])
ax1.set_title('Solution')
ax2.set_title('$|\hat{u}|$');

In [ ]:
uh